In [ ]:
%pip install confluent-kafka avro kafka-python


In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic


admin_client = KafkaAdminClient(
    bootstrap_servers="broker:29092", 
    client_id='test'
)

topic_list = []
topic_list.append(NewTopic(name="avro_schema_topic", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

In [12]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer

# Define the AVRO schema
schema = """
{
  "type": "record",
  "name": "MyRecord",
  "fields": [
    {"name": "id", "type": "int"},
    {"name": "name", "type": "string"}
  ]
}
"""

# Create an AvroProducer with the schema registry configuration
producer = AvroProducer(
    {
        "bootstrap.servers": "broker:29092",
        "schema.registry.url": "http://schema-registry:8081",
    },
    default_value_schema=avro.loads(schema),
)

# Produce a message with the schema
message = {"id": 4, "name": "Gu1y Wolf"}
producer.produce(topic="avro_schema_topic", value=message)
producer.flush()

/tmp/ipykernel_1977/766125364.py:17: DeprecationWarning: AvroProducer has been deprecated. Use AvroSerializer instead.
  producer = AvroProducer(


0

In [13]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroConsumer

# Create an AvroConsumer with the schema registry configuration
consumer = AvroConsumer(
    {
        "bootstrap.servers": "broker:29092",
        "group.id": "test_o4",
        "schema.registry.url": "http://schema-registry:8081",
        "auto.offset.reset": "earliest"
    }
)

# Subscribe to the topic
consumer.subscribe(["avro_schema_topic"])

# Consume messages with the registered schema
running = True
while running:
    message = consumer.poll(5)
    if message is None:
        print("No message")
        running = False
    elif message.error():
        print(f"Error: {message.error()}")
        running = False

    # Get the AVRO-decoded message value
    value = message.value()

    # Access the fields in the value
    id_value = value["id"]
    name_value = value["name"]

    print(f"Received message - ID: {id_value}, Name: {name_value}")

consumer.close()

/tmp/ipykernel_1977/470993943.py:5: DeprecationWarning: AvroConsumer has been deprecated. Use AvroDeserializer instead.
  consumer = AvroConsumer(


Received message - ID: 1, Name: John Doe
Received message - ID: 1, Name: John Doe
Received message - ID: 2, Name: John Wolf
Received message - ID: 2, Name: John Wolf
Received message - ID: 2, Name: John Wolf
Received message - ID: 2, Name: John Wolf
Received message - ID: 2, Name: John Wolf
Received message - ID: 4, Name: Guty Wolf
Received message - ID: 4, Name: Gu2y Wolf
Received message - ID: 4, Name: Gu1y Wolf
No message


AttributeError: 'NoneType' object has no attribute 'error'